In [2]:
import torch
import torch.nn as nn
import scipy.stats as stats
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchviz import make_dot

from icnnet import ICNNet
from mydataset import MyDataset


__toy data__


In [2]:
n = 1000

x = torch.concat([torch.tensor(stats.norm.rvs(loc=-1, scale=0.1, size=(n,1)), dtype=torch.float32), torch.tensor(stats.norm.rvs(loc=1, scale=0.1, size=(n,1)), dtype=torch.float32)])
#y = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])
#c = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])

y = torch.concat([-torch.ones(n), torch.ones(n)])
c = torch.concat([torch.ones(2*n)])

X = torch.stack(torch.unbind(x,dim=0))
Y = torch.stack(torch.unbind(y,dim=0)).unsqueeze(1)
C = torch.stack(torch.unbind(c,dim=0)).unsqueeze(1)

print(X)


tensor([[-0.9689],
        [-1.1237],
        [-0.9792],
        ...,
        [ 0.9013],
        [ 1.0106],
        [ 0.8345]])


__Toy data valou__

In [12]:
n = 1000

x = torch.concat([torch.tensor(stats.norm.rvs(loc=-1, scale=1, size=(2*n,1)), dtype=torch.float32)])
#y = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])
#c = torch.concat([torch.zeros(n//2), torch.ones(n//2), torch.zeros(n//2), torch.ones(n//2)])

c = torch.concat([-torch.ones(n), torch.ones(n)])

X = torch.stack(torch.unbind(x,dim=0))
C = torch.stack(torch.unbind(c,dim=0)).unsqueeze(1)
C = C[torch.randperm(C.size(0))]

Y = C*X
print(X)
print(C)
print(Y)



tensor([[-0.5844],
        [-1.3197],
        [-1.0855],
        ...,
        [-2.0765],
        [-1.8099],
        [-1.5350]])
tensor([[-1.],
        [-1.],
        [ 1.],
        ...,
        [-1.],
        [ 1.],
        [ 1.]])
tensor([[ 0.5844],
        [ 1.3197],
        [-1.0855],
        ...,
        [ 2.0765],
        [-1.8099],
        [-1.5350]])


In [4]:
dataset = MyDataset(X, C, Y)

batch_size = 100  # Example batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

__PICNN training__

In [13]:
from torch import optim
epochs = 50

# Initialize the model
model = ICNNet()
model.train()

# Define the loss function and the optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    for x_batch, c_batch, y_batch in dataloader:
        x_batch.requires_grad_(True)
        c_batch.requires_grad_(True)

        optimizer.zero_grad() # Zero the gradients
        
        output = model(x_batch, c_batch)  # Assuming context c is same as input x

        loss = criterion(output, y_batch) # Compute the loss
        loss.backward() # Backward pass

        optimizer.step() # Update the parameters
        # for layers_k in model.layers_z:
        #     for param in layers_k.parameters():
        #         param.data.clamp_min_(0)
        
        #pass

    #for name, parameter in model.named_parameters():
        #if parameter.requires_grad and parameter.grad is not None:
            #grad_norm = parameter.grad.norm().item()
            #print(f"Gradient norm for {name}: {grad_norm}")

    print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item()}")




Epoch 1/50 Loss: 1.466941475868225
Epoch 2/50 Loss: 1.2606881856918335
Epoch 3/50 Loss: 1.0580757856369019
Epoch 4/50 Loss: 1.3838635683059692
Epoch 5/50 Loss: 1.4151709079742432
Epoch 6/50 Loss: 1.2544455528259277
Epoch 7/50 Loss: 0.9235829710960388
Epoch 8/50 Loss: 1.4019221067428589
Epoch 9/50 Loss: 1.1063801050186157
Epoch 10/50 Loss: 1.2643119096755981
Epoch 11/50 Loss: 1.0063505172729492
Epoch 12/50 Loss: 0.9501168727874756
Epoch 13/50 Loss: 0.9545484781265259
Epoch 14/50 Loss: 1.1857197284698486
Epoch 15/50 Loss: 0.8082815408706665
Epoch 16/50 Loss: 0.9676364064216614
Epoch 17/50 Loss: 1.0840986967086792
Epoch 18/50 Loss: 0.8229528069496155
Epoch 19/50 Loss: 0.9407979846000671
Epoch 20/50 Loss: 1.2312126159667969
Epoch 21/50 Loss: 1.3230749368667603
Epoch 22/50 Loss: 1.2541871070861816
Epoch 23/50 Loss: 1.371496558189392
Epoch 24/50 Loss: 1.2229201793670654
Epoch 25/50 Loss: 0.9114941358566284
Epoch 26/50 Loss: 1.1908451318740845
Epoch 27/50 Loss: 1.135361671447754
Epoch 28/50 L

In [21]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, c_batch, y_batch in dataloader:
        outputs = model(x_batch, c_batch)
        for result, y in zip(outputs, y_batch) :
            print(result, y)
        break



tensor([1.0942]) tensor([1.0662])
tensor([1.3959]) tensor([1.4036])
tensor([0.9848]) tensor([0.9481])
tensor([9.4966e-06]) tensor([-0.7549])
tensor([0.0561]) tensor([0.0483])
tensor([1.8050]) tensor([1.8422])
tensor([0.1343]) tensor([0.1806])
tensor([0.6565]) tensor([0.6473])
tensor([6.6086e-11]) tensor([-1.7745])
tensor([2.0391]) tensor([2.0784])
tensor([6.4235e-17]) tensor([-2.9714])
tensor([0.9973]) tensor([1.0053])
tensor([0.8742]) tensor([0.8353])
tensor([0.0286]) tensor([-0.0306])
tensor([3.2846e-07]) tensor([-1.0424])
tensor([4.2688e-10]) tensor([-1.6136])
tensor([2.6002e-05]) tensor([-0.6688])
tensor([0.0002]) tensor([-0.6140])
tensor([0.3238]) tensor([0.3523])
tensor([1.7604e-10]) tensor([-1.6899])
tensor([0.0015]) tensor([-0.3150])
tensor([4.5149e-07]) tensor([-1.0152])
tensor([1.6314e-17]) tensor([-3.0900])
tensor([1.6247]) tensor([1.6531])
tensor([0.0112]) tensor([-0.1580])
tensor([0.1194]) tensor([0.1523])
tensor([9.7314e-09]) tensor([-1.3444])
tensor([1.6048e-10]) tensor(

Ca apprend pas si bien car c'est bloqué à zéro à cause du relu à la fin

__Makkuva training__

In [ ]:
from torch import optim
epochs = 100
train_freq_g = 20

# Initialize the model
input_size = 1
ICNNf = ICNNet(layer_sizes = [input_size, 2,2,2,2,1], context_layer_sizes=[1,2,2,2,2,2])

output_size = 1
ICNNg = ICNNet(layer_sizes = [output_size, 2,2,2,2,1], context_layer_sizes=[1,2,2,2,2,2])

# Define the loss function and the optimizer
optimizer_f = optim.Adam(ICNNf.parameters())
optimizer_g = optim.Adam(ICNNg.parameters())

for epoch in range(epochs):
    optimizer_f.zero_grad() # Zero the gradients
    optimizer_g.zero_grad() # Zero the gradients
    for _ in range(train_freq_g) :
        for x, c, y in dataloader:
            x.requires_grad_(True)
            y.requires_grad_(True)
            c.requires_grad_(True)

            output_g = ICNNg(y, c)
            diff_output_g = torch.autograd.grad(output_g, y, grad_outputs=torch.ones_like(output_g), create_graph=True)[0]
            
            loss_g = - torch.matmul(y, diff_output_g) -  ICNNf(diff_output_g, c)
            loss_g = torch.mean(loss_g)
            loss_g.backward() # Backward pass

            optimizer_g.step() # Update the parameters
            for layers_k in ICNNg.layers_z:
                for param in layers_k.parameters():
                    param.data.clamp_min_(0)

    for x, c, y in dataloader:
        x.requires_grad_(True)
        y.requires_grad_(True)
        c.requires_grad_(True)


        output_g = ICNNg(y, c)
        diff_output_g = torch.autograd.grad(outputs=output_g, inputs=y, grad_outputs=torch.ones_like(output_g), create_graph=True)[0]

        loss_f = ICNNf(x, c) - ICNNf(diff_output_g, c)
        loss_f = pytorch.mean(loss_f)
        loss_f.backward() # Backward pass
        
        for layers_k in ICNNf.layers_z:
                for param in layers_k.parameters():
                    param.data.clamp_min_(0)


        #testing algorithm
        output_g_test = ICNNg(x, c)
        diff_output_g_test = torch.autograd.grad(output_g_test, x, grad_outputs=torch.ones_like(output_g_test), create_graph=True)[0]

        loss_test = diff_output_g_test-y

        print('y =', y[0], 'diff_output_g =', diff_output_g_test[0])
        
        pass

    print(f"Epoch {epoch+1}/{epochs} Loss: {loss_g.item()}")

y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.2946], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.2056], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.2182], grad_fn=<SelectBackward0>)
y = tensor([0.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.2768], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.3069], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.2117], grad_fn=<SelectBackward0>)
y = tensor([0.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.1833], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.1747], grad_fn=<SelectBackward0>)
y = tensor([1.], grad_fn=<SelectBackward0>) diff_output_g = tensor([-0.1824], grad_fn=<SelectBackward0>)
y = tensor([0.], grad_fn=<SelectBackward0>) diff_output

KeyboardInterrupt: 